In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  # plotting 
import matplotlib.font_manager as fm # to plot the font
from tqdm.auto import tqdm # see progress bar
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont # Darw picture from font
from skimage.transform import resize # Resizing of image
from cv2 import resize as cv2_resize # resizng of image
from keras.preprocessing.image import ImageDataGenerator  # image augmentation on training images ONLY
from sklearn.model_selection import train_test_split  # splitting the data
import keras.backend as K # for custom metrices implementations and other processes that we define
from keras.layers import Dense,BatchNormalization,Input,Dropout,Conv2D,Flatten,MaxPool2D,LeakyReLU # keras layers
from keras.models import Model #Model class
from keras.optimizers import Adam #optimizer
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
# Call backs acts like milestones and if/else while model is being trained
import gc # garbage collector
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/bengaliai-cv19/test_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test_image_data_3.parquet
/kaggle/input/bengaliai-cv19/test_image_data_0.parquet
/kaggle/input/bengaliai-cv19/train.csv
/kaggle/input/bengaliai-cv19/test_image_data_1.parquet
/kaggle/input/bengaliai-cv19/class_map.csv
/kaggle/input/bengaliai-cv19/train_image_data_3.parquet
/kaggle/input/bengaliai-cv19/train_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test.csv
/kaggle/input/bengaliai-cv19/sample_submission.csv
/kaggle/input/bengaliai-cv19/train_image_data_1.parquet
/kaggle/input/bengaliai-cv19/train_image_data_0.parquet


## Inputting data

In [2]:
TRAIN_IMG_PATH = '../input/bengaliai-cv19/train_image_data_' # just for training data
TEST_IMG_PATH = '../input/bengaliai-cv19/test_image_data_'
FILE_TYPE = '.parquet'
final_w = final_h = 64#92 
# these are the final dimensions of an image. We resize the image so that it fits in out memory and uses less
# computational power. You can surely try different dimensions. There is not BEST size

BATCH_SIZE = 128

train_classes = pd.read_csv("../input/bengaliai-cv19/train.csv")

## Model

In [3]:
inputs = Input(shape = (final_w, final_h, 1)) 
# Input layer just takes into account of the size of the input 

model = Conv2D(filters=64, kernel_size=(5,5), padding='SAME', activation='relu')(inputs) 


# These layers are required
model = Flatten()(model)
dense = Dense(192, activation = "relu")(model) 

out1 = Dense(168, activation = 'softmax',name='out_1')(dense) # names of output layers. We need these names
out2 = Dense(11, activation = 'softmax',name='out_2')(dense)  # as they act as the keys for mapping output
out3 = Dense(7, activation = 'softmax',name='out_3')(dense)   # to each later. See in the model.fit()

model = Model(inputs=inputs, outputs=[out1,out2,out3]) # final line of our model construction code
# tell the system only the start and end, it'll find the path

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
R_LR_P = ReduceLROnPlateau(monitor='val_out_1_loss',patience=3,verbose=1,factor=0.5,min_lr=0.00001)
# if validation loss of out_1 is not decreasing for 3 consecutive epochs, decrease the learning rate by 0.5 
# given if learning rate is above 0.00001 and give us little insight on what has happened verbose=1

ES = EarlyStopping(monitor='val_loss',patience=4, min_delta=0.0025,restore_best_weights=True)
# stop the model from fitting data if validation loss has not decreased by 0.0025 in the last 5 epochs and 
# restore best weights for the next time

MCP = ModelCheckpoint('best_model_weight.h5', monitor ='val_loss', verbose =1, 
                      save_best_only = True, save_weights_only=True)
# save the weights in a file name specified only if the validation loss of out_1 layer has improved from 
# last save. out_1 because it's recall matters twice 

callbacks = [R_LR_P,ES,MCP]

In [6]:
def resize_image(df_in,curr_w,curr_h,res_w,res_h,lib='cv2'):
    resized = {} # empty dictonary which stores the values of pixels for each image
    if lib=='skimage':
        for i in range(df_in.shape[0]): # iterate through all the images in dataframe
            image = resize(df_in.loc[df.index[i]].values.reshape(curr_w,curr_h),(res_w,res_h,1))
            # apply resize transformations on per-row basis 
            resized[df_in.index[i]] = image.reshape(-1)  # reshape accordingly
        resized = pd.DataFrame(resized).T # resizing swaps the rows to columns so Transpose sets to default
    
    else: 
        for i in range(df_in.shape[0]):
            image = cv2_resize(df_in.loc[df.index[i]].values.reshape(curr_w,curr_h),(res_w,res_h))
            resized[df_in.index[i]] = image.reshape(-1)
        resized = pd.DataFrame(resized).T 
    return resized

In [7]:
class CustomDataGenerator(ImageDataGenerator):
    def flow(self,x,y=None,batch_size=BATCH_SIZE,shuffle=True,sample_weight=None,seed=None,save_to_dir=None,
             save_prefix='',save_format='png',subset=None): 
        labels_array = None # all the labels array will be concatenated in this single array
        key_lengths = {} 
        # define a dict which maps the 'key' (y1,y2 etc) to lengths of corresponding label_array
        ordered_labels = [] # to store the ordering in which the labels Y were passed in this class
        for key, label_value in y.items():
            if labels_array is None:
                labels_array = label_value 
                # for the first time loop, it's empty, so insert first element
            else:
                labels_array = np.concatenate((labels_array, label_value), axis=1) 
                # concat each array of y_labels 
                
            key_lengths[key] = label_value.shape[1] 
            ordered_labels.append(key)

        for x_out, y_out in super().flow(x, labels_array, batch_size=batch_size):
            label_dict = {} # final dictonary that'll be yielded
            i = 0 # keeps count of the ordering of the labels and their lengths
            for label in ordered_labels:
                target_length = key_lengths[label]
                label_dict[label] = y_out[:, i: i + target_length] 
                i += target_length

            yield x_out, label_dict

In [ ]:
frame = pd.DataFrame({'-1':['File -1']}) # just a random dataframe nothing much
drop_cols = ['image_id','grapheme_root','vowel_diacritic','consonant_diacritic','grapheme'] 
# columns to be dropped after merging with dataframe so that we can have only numerical pixel values

j=-1
#for i in tqdm(range(0,4)):
for i in tqdm(range(0,1)):
    j+=1
    gc.collect() # calling garbage collector to free up memory from last delete
    
    print('Importing file %s'%j)
    df = pd.merge(pd.read_parquet(TRAIN_IMG_PATH+str(j)+FILE_TYPE),train_classes,on='image_id')
    
    grapheme = pd.get_dummies(df['grapheme_root']).values # get the numerical values of classes and convert
    vowel = pd.get_dummies(df['vowel_diacritic']).values  # them to categorical so that [0,1,2] becomes 
    consonant = pd.get_dummies(df['consonant_diacritic']).values # [[1,0,0],[0,1,0],[0,0,1]] where the size of the
    
    print('Dropping Columns.....')
    df.drop(drop_cols,axis=1,inplace=True) # remove all the columnsthat are not of use
    
    print('Resizing....')
    df = (resize_image(df_in=df,curr_w=137,curr_h=236,
                           res_w=final_w,res_h=final_h)/255.).astype('float32')
    
    print('Reshaping into Rank-4..... ')
    df = df.values.reshape(-1, final_w,final_h,1) # reshape into rank-4 matrix 
    
    print('Splitting Train-Test...')
    df, x_test, y_train_graph, y_test_graph, y_train_vowel, y_test_vowel, y_train_conso, y_test_conso =\
    train_test_split(df, grapheme,vowel,consonant, test_size=0.15, random_state=13)
    
    del grapheme
    del vowel
    del consonant
    gc.collect()
    
    print('Fitting into Generator.....')
    aug_generator = CustomDataGenerator(featurewise_center=False,samplewise_center=False,
                                        featurewise_std_normalization=False,samplewise_std_normalization=False,
                                        zca_whitening=False,rotation_range=13, zoom_range = 0.17,
                                        width_shift_range=0.18,height_shift_range=0.16,horizontal_flip=False,
                                        vertical_flip=False) 
    
    aug_generator.fit(df)
    # this will JUST calculate parameters required (PCA, ZCA and others if True) no transformations performed
    
    print('Starting Training........')
    history = model.fit_generator(aug_generator.flow(df, {'out_1': y_train_graph, 'out_2': y_train_vowel, 
                                                     'out_3': y_train_conso}, batch_size=BATCH_SIZE),
                              epochs = 4, validation_data = (x_test, [y_test_graph, y_test_vowel, y_test_conso]),
                              steps_per_epoch=df.shape[0] //128 ,callbacks=callbacks)
    
    stopped_at = ES.stopped_epoch # gives you >=1 at which epoch model stopped due to early stopping
    
    del df
    del x_test
    del y_train_graph
    del y_test_graph
    del y_train_vowel
    del y_test_vowel
    del y_train_conso
    del y_test_conso
    frame[j]='Files completed is'+str(j) # nothing
    frame.to_csv('files.csv') # saving nothing is like committing when offline and preparing a log if crash happens
    gc.collect()

Importing file 0
Dropping Columns.....
Resizing....
Reshaping into Rank-4..... 
Splitting Train-Test...
Fitting into Generator.....
Starting Training........
Epoch 1/4
333/333 [==============================] - 33s 99ms/step - loss: 8.9146 - out_1_loss: 5.0456 - out_2_loss: 2.4495 - out_3_loss: 1.4195 - out_1_accuracy: 0.0256 - out_2_accuracy: 0.2066 - out_3_accuracy: 0.6170 - val_loss: 7.9770 - val_out_1_loss: 4.7074 - val_out_2_loss: 2.0870 - val_out_3_loss: 1.1822 - val_out_1_accuracy: 0.0256 - val_out_2_accuracy: 0.2621 - val_out_3_accuracy: 0.6293

Epoch 00001: val_loss improved from inf to 7.97702, saving model to best_model_weight.h5


In [ ]:
model.load_weights('best_model_weight.h5') 
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for j in range(4):
    print('Processing file %s'%j)
    df = pd.read_parquet(TEST_IMG_PATH+str(j)+FILE_TYPE)
    df.set_index('image_id', inplace=True)
    index_values = df.index.values

    df = (resize_image(df_in=df,curr_w=137,curr_h=236,
                           res_w=final_w,res_h=final_h)/255.).astype('float32')
    df = df.values.reshape(-1, final_w, final_h, 1)
    
    preds = model.predict(df)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(index_values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    
    del df
    gc.collect()

## Writing submission file

In [ ]:
#submission = pd.DataFrame({'row_id': row_id,'target':target},columns = ['row_id','target'])
#submission.to_csv('submission.csv',index=False)
#submission.head()